<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AfDesign (v1.1.1)
Backprop through AlphaFold for protein design.

**WARNING**
1.   This notebook is in active development and was designed for demonstration purposes only.
2.   Using AfDesign as the only "loss" function for design might be a bad idea, you may find adversarial sequences (aka. sequences that trick AlphaFold).

In [ ]:
#@title setup
%time
import os
if not os.path.isdir("params"):
  # download params
  os.system("mkdir params")
  #os.system("apt-get install aria2 -qq")
  os.system("wget https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
import numpy as np

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

# Constrained hallucination: designing an mdm2 binder

For a given protein target and protein binder length, generate/hallucinate a protein binder sequence AlphaFold thinks will bind to the target structure.

To do this, we minimize PAE and maximize number of contacts at the interface and within the binder, and we maximize pLDDT of the binder.
By default, AlphaFold-ptm with residue index offset hack is used. To enable AlphaFold-multimer set: mk_afdesign_model(use_multimer=True).


# Prepare to hallucinate binder

Specify the binding target with `get_pdb(pdb_string)` and the chain by specifying `chain=chain_letter`.

Adjust the number of amino acids in the binder design by specifying `binder_len=number_amino_acids`

In [ ]:
clear_mem()
af_model = mk_afdesign_model(protocol="binder")

# define the target sequence/structure and the desired binder length
af_model.prep_inputs(pdb_filename=get_pdb("1YCR"), chain="A", binder_len=128)

print("target_length",af_model._target_len)
print("binder_length",af_model._binder_len)
print("weights",af_model.opt["weights"])


# Run hallucination

**Stage 1** starts by applying gradient descent to the sequence logits directly, freely allowing these values to range to very large and even negative magnitudes. Stage 1 gradually switches to gradient descent on softmax(logits)

**Stage 2** starts with softmax(logits) and gradually switches to hardmax(logits)

**Stage 3** finalizes the design on hardmax(logits)

```
af_model.design_3stage(stage_1_steps, stage_2_steps, stage_3_steps)
```

In [ ]:
af_model.restart()
%time
af_model.design_3stage(100,100,10)

# Save design structure

In [ ]:
af_model.save_pdb(f"./mdm2_binder.pdb")

# Plot results

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.plot_pdb()

# Show design sequence

In [ ]:
af_model.get_seqs()